<a href="https://colab.research.google.com/github/Jeisha/FYP-Temp/blob/main/PART_2_Licence_Plate_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('drive/MyDrive/FYP')

Mounted at /content/drive


In [ ]:
CUSTOM_MODEL_NAME = 'my_ssd_mobilenet' 
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'ANNOTATION_PATH': os.path.join('Model', 'Lisence Detector','annotations'),
    'CHECKPOINT_PATH': os.path.join('Model', 'Lisence Detector',CUSTOM_MODEL_NAME), 
    'REALTIME_TEST_PATH':os.path.join('realtime test'),
    'DATA_COLLECTION_PATH':os.path.join('datacol'),
    'IMAGE_WORK_PATH': os.path.join('image_work'),
    'CAR_DETECTED_PATH':os.path.join('image_work','CAR_DETECTED'),
    'PLATE_DETECTION_PATH':os.path.join('image_work','PLATE_DETECTION'),
    'PLATE_DETECTED_PATH':os.path.join('image_work','PLATE_DETECTED')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Model', 'Lisence Detector', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# Install Tensorflow Object Detection 
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 62 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/drive/MyDrive/FYP/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.9 MB 29.8 MB/s 
     |████████████

In [ ]:
!pip install "opencv-python-headless<4.3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.6 MB 55.1 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


In [ ]:
import numpy as np
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.builders import model_builder
from object_detection.utils import config_util
import glob
import matplotlib.pyplot  as plt
import cv2
%matplotlib inline

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-3')).expect_partial()

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
def rescale_frame_plate_detection(frame):
    width = 1080
    height = 720
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)[300:1080,300:800]

In [ ]:
plate = 0
plate_d = 0
while True:
    car_paths = []
    for i in sorted(glob.glob(os.path.join(paths['CAR_DETECTED_PATH'],'*.png'))):
        car_paths.append(i)

    for path in car_paths:
        img = cv2.imread(path)
        try:
            image = rescale_frame_plate_detection(img)
            image_np = np.array(image)
            wi,he,c = image_np.shape
            input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
            detections = detect_fn(input_tensor)
            
            num_detections = int(detections.pop('num_detections'))
            detections = {key: value[0, :num_detections].numpy()
                        for key, value in detections.items()}
            detections['num_detections'] = num_detections

            # detection_classes should be ints.
            detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

            label_id_offset = 1
            image_np_with_detections = image_np.copy()
            
            maxScore_index = 0
            maxScore = 0
            sameMax = 0

            for i in range(len(detections['detection_classes'])):
                if detections['detection_scores'][i] > maxScore:
                    maxScore_index = i
                    maxScore = detections['detection_scores'][i]
                    sameMax = 0
                elif detections['detection_scores'][i] == maxScore:
                    sameMax += 1
            if sameMax > 0:
                print(f'{sameMax} on {plate}')
            if maxScore > .85:
                x,y,w,h = detections['detection_boxes'][maxScore_index]
                x,y,w,h = int(x*wi),int(y*he),int(w*wi),int(h*he)
                roi = image_np_with_detections[x:w-5,y+5:h-5]
                # x = atas
                # w = bawah
                # y = kiri
                # h = kanan
                img_item = os.path.join(paths['PLATE_DETECTED_PATH'],f'plate-{plate:05d}.png')
                cv2.imwrite(img_item, roi)
                plate +=1
            
            img_item = os.path.join(paths['PLATE_DETECTION_PATH'],f'plate-{plate:05d}({plate_d:05d}).png')
            cv2.imwrite(img_item, image_np_with_detections)
            plate_d +=1
            os.remove(path)
        except:
            print(f'Error detected, break. Next Loop')
            break

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-06a5517ba360>", line 5, in <module>
    for i in sorted(glob.glob(os.path.join(paths['CAR_DETECTED_PATH'],'*.png'))):
  File "/usr/lib/python3.7/glob.py", line 20, in glob
    return list(iglob(pathname, recursive=recursive))
  File "/usr/lib/python3.7/glob.py", line 72, in _iglob
    for name in glob_in_dir(dirname, basename, dironly):
  File "/usr/lib/python3.7/glob.py", line 80, in _glob1
    names = list(_iterdir(dirname, dironly))
  File "/usr/lib/python3.7/glob.py", line 122, in _iterdir
    for entry in it:
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_trac

KeyboardInterrupt: ignored